# Fractional Cover

## This notebook is currently under development!

## Notebook Summary

This notebook provides a fractional cover product which can be used to represent land cover. 

We're utilising Landsat 8 on AWS data https://docs.opendata.aws/landsat-pds/readme.html, and have indexed into an instance of the ODC, for more information about ODC please visit our landing page Landing Page.

The basis for this notebook below and the following explanation are from Digital Earth Australia http://geoscienceaustralia.github.io/digitalearthau/notebooks/02_DEA_datasets/Introduction_to_Fractional_Cover.html.

### What is Fractional Cover

Fractional Cover represents the proportion of the land surface that is bare (BS), covered by photosynthetic vegetation (PV), or non-photosynethic vegetation (NPV).

The Fractional Cover product was generated using the spectral unmixing algorithm developed by the Joint Remote Sensing Research Program (JRSRP) which used the spectral signature for each pixel to break it up into three fractions, based on field work that determined the spectral characteristics of these fractions. The fractions were retrieved by inverting multiple linear regression estimates and using synthetic endmembers in a constrained non-negative least squares unmixing model.

The green (PV) fraction includes leaves and grass, the non-photosynthetic fraction (NPV) includes branches, dry grass and dead leaf litter, and the bare soil (BS) fraction includes bare soil or rock.

### Fractional Cover Bands

Bare Soil (bare ground, rock, disturbed) (BS): - Bare Ground (bare soil, rock) percentage; Digital Number 10000 = 100%

Photosythetic Vegetation. (green grass, trees, etc.) (PV): - Photosynthetic Vegetation: Green Vegetation percentage;Digital Number 10000 = 100%

Non-Photosythetic vegetation (litter, dead leaf and branches) (NPV): - Non-Photosynthetic Vegetation (litter, dead leaves andbranches) percentage; Digital Number 10000 = 100%

Unmixing Error (UE): - Unmixing Error. The residual error, defined as the Euclidean Norm of the Residual Vector. High values express less confidence in the fractional components.

The initial instance of this product provides fractional cover of a single L8 image with cloud and water removal. Additional iterations will incorporate a time range for which the median product will be used to reduce the influence of cloud on the product.

Additionally a change product will be produced to allow for areas where changes in land cover have been experienced to be identified. 


### Import required modules

In [1]:
%matplotlib inline

import datacube
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from datacube.storage import masking

import sys
sys.path.append('/home/jovyan/odc-hub/')

## Initialise

In [2]:
dc = datacube.Datacube(app='ls8-fcscene')

## Set up datacube query.
This includes options for crs, a groupby for overlapping datasets, a time range, and a spatial extent. 

In [3]:


# You can specify a specific Sentinel 2 MGRS location using region code:
# query['region_code']='089079'
# alternatively you can create a bounding box, to do this you need three parameters:

#set baseline start and end of period
baseline_start_date = '2015-3-1'
baseline_end_date = '2015-9-1'

res = (200)

from utils_sac.createAOI import create_lat_lon
aoi_wkt = "POLYGON ((177.62557983398438 -17.590848708679893, 177.77372360229492 -17.590848708679893, 177.77372360229492 -17.488875828028657, 177.62557983398438 -17.488875828028657, 177.62557983398438 -17.590848708679893))"

output_projection = "EPSG:32760"


In [4]:
#time_range
#format dates
from datetime import datetime
def createDate(inputStart, inputEnd):
    start = datetime.strptime(inputStart, '%Y-%m-%d')
    end = datetime.strptime(inputEnd, '%Y-%m-%d')
    startDates = start.date()
    endDates = end.date()
    time_period = (startDates, endDates)
    return time_period
baseline_time_period = createDate(baseline_start_date, baseline_end_date)
#create resolution
resolution = (-res, res)

In [5]:
# format area
lat_extents, lon_extents = create_lat_lon(aoi_wkt)
print("Lat:", lat_extents, "\n"
      "Lon:", lon_extents)

Lat: (-17.590848708679893, -17.488875828028657) 
Lon: (177.62557983398438, 177.77372360229492)


## Load Data 

In [6]:
query = {
    'x': lon_extents,
    'y': lat_extents,
    'output_crs': output_projection,
    'resolution': resolution,
    'time': baseline_time_period,
}

In [ ]:
#Load rgb from L8 and pixel quality band, and display the image.  
landsat_ds = dc.load(
    platform = 'LANDSAT_8',
    product = 'ls8_usgs_sr_scene',
    measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa'],
    **query
)

In [ ]:
from utils_sac.clean_mask import landsat_qa_clean_mask
platform = 'LANDSAT_8'
cloud_mask = landsat_qa_clean_mask(landsat_ds, platform)
#land_mask = landsat_qa_clean_mask(landsat_ds, platform, cover_types=['clear'])
land_and_water_dataset = landsat_ds.where(cloud_mask)
#land_dataset = qamasks.where(land_mask)

In [ ]:
from utils_sac.dc_mosaic import create_max_ndvi_mosaic, create_median_mosaic, create_mosaic

In [ ]:
land_composite = create_median_mosaic(land_and_water_dataset, cloud_mask)
mosaic_masked = landsat_qa_clean_mask(land_composite, platform)

In [ ]:
mosaic_masked.plot()

In [ ]:
from utils_dcal.data_cube_utilities.dc_rgb import rgb
rgb(land_composite, x_coord='x', y_coord='y', use_data_min=True, use_data_max=True)
#land_composite.red.plot(cmap = "Greens")

In [ ]:
from utils_dcal.data_cube_utilities.dc_fractional_coverage_classifier import frac_coverage_classify 
#land_composite = land_composite.rename({"x":"longitude", "y":"latitude"})
frac_classes = frac_coverage_classify(land_composite, clean_mask = mosaic_masked.values)
frac_classes = frac_classes.where(mosaic_masked != 0)


In [ ]:
print(frac_classes)

In [ ]:
#load the bands of the fractional cover product. 
#full_resolution = dc.load(
#    measurements=['PV','BS','NPV','UE'],
#    product = 'ls8_usgs_fc_scene',
#    **query #use the query we defined above
#)

## Plot Fractional Cover and Unmixing Error bands

The Unmixing Error is high over water. 

In [ ]:
#plot the fractional cover bands. 
scene = 0
plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
frac_classes.pv.plot(cmap='gist_earth_r')
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
frac_classes.bs.plot(cmap='Oranges')
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
frac_classes.npv.plot(cmap='copper')
ax3.set_title('NPV')



plt.tight_layout()
plt.show()

### To remove the error, the UE band can be filtered - although WOfS provides a more robust mask. 

In [ ]:
#this image shows that you can filter using unmixing error if you wish
#plt.figure(figsize=(12,8))
#full_resolution.UE.where(full_resolution.UE<=40.0).isel(time=scene).plot(cmap='gist_earth_r')
#ax1.set_title('UE filtered')

## Filter Fractional cover scenes using WOfS feature layers (WOFLs)

WOfS provides a scene by scene mask to remove areas of water. Additional it provides cloud and cloud shadow masks. 

## HERE ADD in water_classification function

In [ ]:
#Load in water classification.

water_scenes = dc.load(product="water_classification",
               group_by='solar_day',
               measurements = ["water"],
               **query)


In [ ]:
# change cloud to no data value
#clearsky_scenes = scenes.where(scenes != -9999)

In [ ]:
#water_classes = wofs_classify(scenes, clean_mask=clearsky_masks.values , no_data = np.nan , x_coord='x', y_coord = "y")

## Create mask layers based on WOfS.

Masks for to remove undesired features from the fractional cover product - clouds, cloud-shadow and water. 

In [ ]:
clearwofl = masking.make_mask(wofls, dry=True)
wetwofl = masking.make_mask(wofls, wet=True)
cloudwofl = masking.make_mask(wofls, cloud=True)
cloudshwofl = masking.make_mask(wofls, cloud_shadow=True)


In [ ]:
#Plot showing each of the created masks. 
scene = 0
plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
wetwofl.water.isel(time=0).plot()
ax1.set_title('Wet')

ax2=plt.subplot(gs[1,0])
cloudshwofl.water.isel(time=0).plot()  
ax2.set_title('Cloud Shadow')

ax3=plt.subplot(gs[0,1])
cloudwofl.water.isel(time=0).plot()  
ax3.set_title('Cloud')

ax4=plt.subplot(gs[1,1])
clearwofl.water.isel(time=0).plot()  
ax4.set_title('Clear and Dry')

plt.tight_layout()
plt.show()

In [ ]:
#match WOFL times to our fractional cover times
unwofld = full_resolution.where(full_resolution.time == wetwofl.time)
#unwofld = full_resolution.where(full_resolution.output_crs == wetwofl.output_crs)
#mask out water, clouds and cloudshadow from fractional cover
unwofld = unwofld.where(wetwofl.water==False).where(cloudwofl.water==False).where(cloudwofl.water==False)
#plot masked PV> 
unwofld.PV.isel(time=0).plot()

## Create a plot showing FC components.

#Current mask issues in south west corner - with the water mask. 

In [ ]:
# Plot fractional cover as cloud free RGB image
###work out how to plot this for single image and also how to make a mosaic. 
unwofld[['BS','PV','NPV']].to_array().plot.imshow(
    col='time',
    figsize=(12, 8),
    vmin=0,
    vmax=3000
);

#clearwofl.water.isel(time=0).plot() 